# API Mini Project
****
### Introduction
In this exercise we will be pulling some data from the Quandl API. Qaundl is currently the most widely used aggregator of financial market data.

We will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery. The company is listed under the stock ticker AFX_X.

Note: we will only be using the Python standard library along with requests, json and numpy

In order to access your own API KEY the following steps must be taken:

1. Register a free account on the http://www.quandl.com website.
2. Access your unique API key and stare it into the variable API_KEY

In [1]:
# Import relevant modules
import requests
import json
import numpy as np
import operator
from collections import defaultdict, OrderedDict

In [2]:
# Store the API key as a string. Enter your personal API to make the script work.
API_KEY = ' ' 

In [3]:
# Call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X?start_date=2019-05-17&end_date=2019-05-17&api_key=' + API_KEY
r = requests.get(url)
json_data = r.json()

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
json_data

{'dataset': {'id': 10095370,
  'dataset_code': 'AFX_X',
  'database_code': 'FSE',
  'name': 'Carl Zeiss Meditec (AFX_X)',
  'description': 'Stock Prices for Carl Zeiss Meditec (AFX) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  'refreshed_at': '2019-05-20T22:30:41.159Z',
  'newest_available_date': '2019-05-20',
  'oldest_available_date': '2000-06-07',
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'frequency': 'daily',
  'type': 'Time Series',
  'premium': False,
  'limit': None,
  'transform': None,
  'column_index': None,
  'start_date': '2019-05-17',
  'end_date': '2019-05-17',
  'data': [['2019-05-17',
    None,
    89.4,
    88.55,
    89.25,
    None,
    117480.0,
    10466561.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None,
  'database_id': 6129}}

The structure of the object "json_data" is a nested dictionary. Within the 'dataset' key you have a bunch of key-value pairs that are accessible for various information. For the purposes of this exercise we are interested in looking at 'column_names' and 'data'. 'column_names' has a list of each column name and 'data' has a nested list for from each day of collected data.

## Task 
****
#### 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [5]:
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X?start_date=2017-01-01&end_date=2017-12-31&api_key=' + API_KEY
# Request dat from url and API Key
r = requests.get(url)
r

<Response [200]>

#### 2. Convert the returned JSON object into a Python dictionary.
+ From looking at the structure for one day, we know that we want the column_names and data to be attached in a dictionary. I have two methods below to do so.


In [6]:
# Convert JSON Object into a dictionary
afx_data = r.json()['dataset']

METHOD 1 - Collections package

In [7]:
column_list = afx_data['column_names']    # Separate column names into new list
data_list = afx_data['data']              # Separate data into new list
afx = defaultdict(list)                   # Create new dict

# Append data values to appropriate column key
for d in data_list:
    for x in range(len(column_list)):
        afx[column_list[x]].append(d[x])

METHOD 2 - Zip

In [8]:
# Append data values to appropriate column key
afx = dict(zip(afx_data['column_names'], list(zip(*afx_data['data']))))

#### 3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [9]:
high_open = max(o for o in afx['Open'] if o != None)
low_open = min(l for l in afx['Open'] if l != None)

print('High Opening Price: ' + str(high_open))
print('Low Opening Price: ' + str(low_open))

High Opening Price: 53.11
Low Opening Price: 34.0


#### 4. What was the largest change in any one day (based on High and Low price)?

In [10]:
change = max(abs(x - i) for (x, i) in zip(afx['High'], afx['Low']))
print('The largest change in one day was: ' + str(change) + '.')

The largest change in one day was: 2.8100000000000023.


#### 5. What was the largest change between any two days (based on Closing Price)?

In [11]:
two_change = max(abs(x-i) for (x,i) in zip(afx['Close'], afx['Close'][1:]))
print('The largest change between any two days based off closing price was: ' + str(two_change) + '.')

The largest change between any two days based off closing price was: 2.559999999999995.


#### 6. What was the average daily trading volume during this year?

In [12]:
avg_trade_vol = sum(afx['Traded Volume']) / len(afx['Traded Volume'])
print('The average daily trading volume was: ' + str(avg_trade_vol) + '.')

The average daily trading volume was: 89124.33725490196.


#### 7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [13]:
# Function to find the median of a list
def find_median(values):
    # Sort the list if it isn't already
    sorted_values = sorted(values)
    # Determine the size of the list
    size = len(values)
    # Determine the middle index of any list
    index = (size - 1) // 2
    
    # If list has even number of inputs take the two middle values and average them
    # Elif they are an odd number of inputs, use the middle value as the median.
    # Else return an error for not enough values to calculate median
    if size % 2 == 0:
        return (sorted_values[index] + sorted_values[index + 1])/2
    elif size % 2 != 0:
        return sorted_values[index]
    else:
        raise ValueError('Not enough values to calculate median.')    
        
print('The median of trading volume was: ' + str(find_median(afx['Traded Volume'])) + '.')

The median of trading volume was: 76286.0.
